In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

# Change to MPS logic if on mac
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

For this notebook, use the hints to fill in the missing code.

In [2]:
FILL_IN = "FILL IN"

In [3]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [5]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    filename = f'/content/eng-fra-1.txt'
    lines = open(filename, encoding='utf-8').read().strip().split('\n')


    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [6]:
MAX_LENGTH = 10

# We'll just read in data with these prefixes so we have easier data to deal with
# These are "target" prefixes
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

# Only use pairs where the english data (pair[1]) has the prefix above
# Also, only consider data where pair[0] and pair[1] have length less than MAX_LENGTH
# Split on space first here also as MAX_LENGTH means the number of tokens (words), not number of chars
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
       len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [7]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['vous etes consciencieuse .', 'you re conscientious .']


In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # Initialize the embedding and lstm; use batch_first=True
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)

    def forward(self, input, hidden_cell):
        # Get the embeddings and reshape to be (1, 1, -1)
        # Why? remember we use batch size = 1 in this HW for simplicity
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        # Push through the lstm
        output, hidden_cell = self.lstm(output, hidden_cell)
        return output, hidden_cell

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [9]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # Initialize the embedding
        self.embedding = nn.Embedding(output_size, hidden_size)
        # Initialize an LSTM with yt and kt dimensions hidden_size
        # Use batch_first=True
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        # Initialize a Linear layer going to the appropriate vocabulary size
        self.out = nn.Linear(hidden_size, output_size)
        # Optional: If you use NLLLoss, initialize LogSoftmax here
        # What dimension?
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden_cell):
        output = self.embedding(input).view(1, 1, -1)
        # Push through relu
        output = F.relu(output)
        # Push output and hidden_cell tuple through the lstm
        output, hidden_cell = self.lstm(output, hidden_cell)
        # Apply LogSoftmax to output (?)
        # Note you can leave this out if you wor with logits and CrossEntropyLoss
        output = self.softmax(self.out(output[0]))
        return output, hidden_cell

    def initHidden(self):
        # Used to initialize the hidden state (or cell state) to a tensor of dimension (1, 1, hidden_size)
        # Just return a tensor here
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [10]:
# Split a sentence by ' ' and return a list of the tokens (int ids) for each word
# Use word2index

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]
# Call the above on a sentence
# After calling, add the EOS_token (int id) to the gotten list
# Return a tensor, but reshape it so it's dimensions (-1, 1)
def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)
# For a source, target pair, call the above. Return a tuple of 2 tensors, one input_te
def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [11]:
teacher_forcing_ratio = 0.5

def train(
    input_tensor,
    target_tensor,
    encoder,
    decoder,
    encoder_optimizer,
    decoder_optimizer,
    criterion,
    max_length=MAX_LENGTH
):
    # Initialize the hidden and cell states
    encoder_hidden = encoder.initHidden()
    encoder_cell = encoder.initHidden()

    # Reset the optimizer gradients to 0
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    loss = 0

    # Reverse the input here, see below
    input_tensor = input_tensor.flip(0)

    for it in range(input_length):
        encoder_output, (encoder_hidden, encoder_cell) = encoder(
            input_tensor[it], (encoder_hidden, encoder_cell)
        )

    # Initialize the decoder input to the SOS_token
    decoder_input = torch.tensor([[SOS_token]], device=device)
    # Initialize the hidden states of the decoder with the hidden states of the encoder
    decoder_hidden = encoder_hidden
    decoder_cell = encoder_cell

    # For this pair, use teacher forcing with 50% probability, else don't
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    target_length_used = 0

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        target_length_used = target_length
        for jt in range(target_length):
            decoder_output, (decoder_hidden, decoder_cell) = decoder(
                decoder_input, (decoder_hidden, decoder_cell)
            )
            loss += criterion(decoder_output, target_tensor[jt])
            decoder_input = target_tensor[jt]  # Teacher forcing
    else:
        # Without teacher forcing: use its own predictions as the next input
        for jt in range(target_length):
            decoder_output, (decoder_hidden, decoder_cell) = decoder(
                decoder_input, (decoder_hidden, decoder_cell)
            )
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # Detach from history as input
            loss += criterion(decoder_output, target_tensor[jt])
            target_length_used += 1

            # If the EOS_token was generated, exit
            if decoder_input.item() == EOS_token:
                break

    # Collect gradients
    loss.backward()

    # Do a step; do this both for the encoder and the decoder
    encoder_optimizer.step()
    decoder_optimizer.step()

    # Return the loss for this pair. Note that you'll need to use target_length_used not target_length; why?
    return loss.item() / target_length_used


In [12]:
# Helper functions used to time the optimizations
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

trainIters below does SGD (batch 1) on the data.

In [13]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)


def trainIters(
    encoder,
    decoder,
    n_iters,
    print_every=1000,
    plot_every=100,
    learning_rate=0.01
):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    # Initialize the encoder and decoder optimizers with the above learning rate
    encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)

    # Get n_iters training pairs
    # In this example, we are effectively doing SGD with batch size 1
    training_pairs = [tensorsFromPair(random.choice(pairs)) for _ in range(n_iters)]

    # The loss; either NLLLoss if you use log sigmoids or CrossEntropyLoss if you use logits
    criterion = nn.CrossEntropyLoss()

    for it in range(1, n_iters + 1):
        training_pair = training_pairs[it - 1]
        input_tensor, target_tensor = training_pair

        # Train on the input, target pair
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)

        # Update the total loss and the plot loss
        # We can plot and print at different granularities
        print_loss_total += loss
        plot_loss_total += loss

        if it % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print(
                '%s (%d %d%%) %.4f' % (
                    timeSince(start, it / n_iters),
                    it, it / n_iters * 100, print_loss_avg)
            )

        if it % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

            showPlot(plot_losses)

Fill in the evaluation function using the hints below.

In [14]:
# For a certain input, get the predicted output sentence
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        # Transform the input sentence into a tensor
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        # Initilize the hidden and cell states of the LSTM
        encoder_hidden = encoder.initHidden()
        encoder_cell = encoder.initHidden()

        # Run the data through the LSTM word by word manually
        # At each step, feed in the input, the hidden state, and the cell state and calture the new hidden / cell states
        for it in range(input_length):
            encoder_output, (encoder_hidden, encoder_cell) = encoder(input_tensor[it].view(1, 1), (encoder_hidden, encoder_cell))


        # Initialize the decoder input with the SOS token
        # This is y_0
        decoder_input = torch.tensor([[SOS_token]], device=device)

        # Initialize the decoder hidden and cell states with the final encoder hidden and cell states
        decoder_hidden = encoder_hidden
        decoder_cell = encoder_cell

        decoded_words = []

        for jt in range(max_length):
            # As with the encoder run the \hat{y}_{t-1}, hidden, and state cells through the decoder
            # Capture the new hidden and cell states and the logits or log_softmax for the vocabulary
            decoder_output, (decoder_hidden, decoder_cell) = decoder(decoder_input, (decoder_hidden, decoder_cell))
            # Get the top y for the decoder, this will be the new \hat{y}_t which we can use at the next step
            _, topi = decoder_output.data.topk(1)

            decoded_words.append(output_lang.index2word[topi.item()])
            if topi.item() == EOS_token:
             break

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [15]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_cell = torch.zeros_like(encoder_hidden)  # Assuming similar structure for cell state

        # Encoding
        for ei in range(input_length):
            _, (encoder_hidden, encoder_cell) = encoder(input_tensor[ei], (encoder_hidden, encoder_cell))

        # Decoding
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden
        decoder_cell = encoder_cell
        decoded_words = []

        for di in range(max_length):
            decoder_output, (decoder_hidden, decoder_cell) = decoder(decoder_input, (decoder_hidden, decoder_cell))
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words


In [16]:
hidden_size = 256
cell_size = 256

# Assuming input_size and output_size are defined as the sizes of your vocabularies
input_size = len(input_lang.word2index)  # Number of words in the input language vocabulary
output_size = len(output_lang.word2index)  # Number of words in the output language vocabulary

encoder = EncoderRNN(input_size, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_size).to(device)



trainIters(encoder, decoder, 75000, print_every=5000)

<ipython-input-13-cfd37c547424>:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


1m 18s (- 18m 17s) (5000 6%) 3.8765


RuntimeError: cuDNN error: CUDNN_STATUS_MAPPING_ERROR

Here we will do an evaluation.
Gather up n=7500 random pairs and for each pair get te BLEU score.
For this exercise, don't use BLUE based on 4-grams, use just 2-grams (you need to figure out how to specify this in $sentence\_bleu$ below).
Investigate how this can be done with "sentence_bleu" in nltk - you can also find other implementations.
Print the average BLEU score after you've randomly drawn the sentences. You should be a BLEU of about 10 or slightly more.
Note that is is training BLEU, which is all I'd like you to get.

In [ ]:
from nltk.translate.bleu_score import sentence_bleu


def evaluateRandomly(encoder, decoder, pairs, n=7500, debug=False):
    bleu_scores = []
    for i in range(n):
        # Randomly choose a pair of sentences
        pair = random.choice(pairs)
        if debug:
            print('French Original: ', pair[0])
            print('English Reference: ', pair[1])

        # Run the source French sentence through the encoder-decoder and get the output
        output_words = evaluate(encoder, decoder, pair[0])
        # If <EOS> is at the end of output_words, remove it
        if output_words[-1] == '<EOS>':
            output_words.pop()

        output_sentence = ' '.join(output_words)

        # Get the BLEU score based on 1 and 2 grams (words, bigrams); use 50% weight on each
        score = 100.0 * sentence_bleu([pair[1].split(' ')], output_words, weights=(0.5, 0.5))

        # Append the BLEU score you got to the list of BLEU scores you keep
        bleu_scores.append(score)

        if debug:
            print('Candidate Translation: ', output_sentence)
            print('BLEU: ', score)
            print('')

    # Return the mean of the BLEU scores
    print('The mean BLEU score is: ', np.mean(bleu_scores))

# Assumed required parameters or modules are available, like evaluate function and others.


In [ ]:
# BLEU > 10 % is expected here
evaluateRandomly(encoder, decoder)